In [1]:
# https://stackoverflow.com/questions/45741878/using-binary-crossentropy-loss-in-keras-tensorflow-backend
# https://www.machinecurve.com/index.php/2019/10/22/how-to-use-binary-categorical-crossentropy-with-keras/

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.model_selection import train_test_split
from keras.callbacks import CSVLogger
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%load_ext tensorboard
import datetime

#tf.config.experimental.list_physical_devices('GPU')

# RNN based Models (RNN & LSTM & GRU) using tensorflow keras

In [2]:
!rm -rf ./logs/RNN #dont run this line unless you want to delete the tensorboard logs

In [3]:
#normalize X values to help model converge
def normalize(data):
    mean = np.mean(data, axis=0)
    std = np.std(data, axis=0)
    data = (data - mean) / std
    return data

In [4]:
def convert_3D(matrix):
    return np.expand_dims(matrix, -1)

In [5]:
#retrieve and format data - into labels and examples from the dataset
def features_and_labels(filename_a, filename_b):
    with open(filename_b, 'rb') as f:
        labels = np.load(f).ravel()
    
    data = pd.read_pickle(filename_a)
    #https://stackoverflow.com/questions/22798934/pandas-long-to-wide-reshape-by-two-variables
    data_x = data.pivot(index='id', columns='time', values='FLUX')
    data_x = data_x.to_numpy()
    data_x = convert_3D(normalize(data_x))
    return data_x, labels

In [6]:
training_data_x, training_data_y = features_and_labels("../Kepler-Train.pkl", "../Labels-Train.npy")
testing_data_x, testing_data_y = features_and_labels("../Kepler-Test.pkl", "../Labels-Test.npy")
validation_data_x, validation_data_y = features_and_labels("../Kepler-Val.pkl", "../Labels-Val.npy")

In [7]:
RNNmodel = Sequential([
    SimpleRNN(32, input_shape=(training_data_x.shape[1:]), activation='relu', return_sequences=True),
    SimpleRNN(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

RNNmodel.compile(
    loss=BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.00001),
    metrics=["accuracy", tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.FalseNegatives(),
             tf.keras.metrics.FalsePositives(), tf.keras.metrics.TrueNegatives(), tf.keras.metrics.TruePositives()]
)

log_dir = "logs/RNN/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

csv_logger = CSVLogger('keras-models/KerasRNN/training.csv',separator=",", append=False)

RNNmodel.fit(training_data_x, training_data_y, epochs=3, validation_data=(validation_data_x,validation_data_y), callbacks=[tensorboard_callback, csv_logger])

model = RNNmodel
model.save('keras-models/KerasRNN')

Epoch 1/3
394/394 [==============================] - 112s 277ms/step - loss: 0.6647 - accuracy: 0.6209 - auc: 0.5547 - precision: 0.1883 - recall: 0.1975 - false_negatives: 2316.0000 - false_positives: 2457.0000 - true_negatives: 7246.0000 - true_positives: 570.0000 - val_loss: 0.6244 - val_accuracy: 0.7332 - val_auc: 0.6154 - val_precision: 0.1358 - val_recall: 0.0305 - val_false_negatives: 350.0000 - val_false_positives: 70.0000 - val_true_negatives: 1143.0000 - val_true_positives: 11.0000
Epoch 2/3
394/394 [==============================] - 100s 255ms/step - loss: 0.5679 - accuracy: 0.7611 - auc: 0.6871 - precision: 0.0643 - recall: 0.0031 - false_negatives: 2877.0000 - false_positives: 131.0000 - true_negatives: 9572.0000 - true_positives: 9.0000 - val_loss: 0.5039 - val_accuracy: 0.7706 - val_auc: 0.7313 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_negatives: 361.0000 - val_false_positives: 0.0000e+00 - val_true_negatives: 1213.0000 - val_true_positives: 0.0000

In [8]:
%tensorboard --logdir logs/RNN/fit